In [1]:
import pandas as pd
import geopandas as gpd

In [6]:
def calc_h3_cell_beach_slopes(h3_gdf, slope_gdf):
    """
    Calculate the average beach slope for every row in hr5_cells using beach_slope stored in slope_gdf.

    Parameters:
    hr5_cells (GeoDataFrame): GeoDataFrame containing HR5 cells.
    slope_gdf (GeoDataFrame): GeoDataFrame containing beach slopes.

    Returns:
    GeoDataFrame: GeoDataFrame with average beach slopes for each HR5 cell.
    """
    # Perform spatial join
    joined_gdf = gpd.sjoin(h3_gdf, slope_gdf, how="left", predicate="intersects")

  # Group by HR5 cell and calculate the average beach slope
    avg_slope_gdf = joined_gdf.groupby(joined_gdf.index).agg({'beach_slope': 'mean'}).reset_index()
    avg_slope_gdf['beach_slope'] = avg_slope_gdf['beach_slope'].round(3)

    # Merge the average slopes back to the original hr5_cells GeoDataFrame
    h3_gdf = h3_gdf.merge(avg_slope_gdf, left_index=True, right_on='index')

    h3_gdf.drop(columns=['index', 'index_y'], inplace=True)
    h3_gdf.rename(columns={'index_x': 'cell_id'}, inplace=True)

    return h3_gdf


In [7]:
hr5_cells = gpd.read_file('global-inputs/HR5-change-cells-aoi.gpkg')
hr5_cells.to_crs(crs='EPSG:4326', inplace=True)

slope_gdf = gpd.read_file('global-inputs/transects_edit.geojson')
slope_gdf = slope_gdf.clip(hr5_cells)
slope_gdf = gpd.GeoDataFrame(slope_gdf[['id', 'beach_slope', 'geometry']])
slope_gdf.dropna(inplace=True)

hr5_cells.to_crs("EPSG:2193", inplace=True)
slope_gdf.to_crs("EPSG:2193", inplace=True)

hr5_cells = calc_h3_cell_beach_slopes(hr5_cells, slope_gdf)


In [8]:
hr5_cells

,cell_id,parent_id,sediment_area_ha,geometry,beach_slope
0,85bb0b37fffffff,84bb0b3ffffffff,88.52,"POLYGON ((1922896.542 5807892.773, 1918824.379...",0.066
1,85bb56cffffffff,84bb56dffffffff,98.72,"POLYGON ((1895219.978 5828219.058, 1891134.011...",0.062
2,85bb560ffffffff,84bb561ffffffff,33.08,"POLYGON ((1869065.502 5866362.246, 1864965.364...",0.076
3,85bb5293fffffff,84bb529ffffffff,42.80,"POLYGON ((1858357.096 5912132.056, 1854249.225...",0.098
4,85bb0b07fffffff,84bb0b1ffffffff,53.76,"POLYGON ((1950490.925 5787619.369, 1946432.877...",0.055
...,...,...,...,...,...
317,85bb2d37fffffff,84bb2d3ffffffff,7.20,"POLYGON ((1670719.738 5326592.803, 1666569.495...",0.061
318,85bb2d33fffffff,84bb2d3ffffffff,54.52,"POLYGON ((1684781.284 5316739.357, 1680639.31 ...",0.054
319,85bb53affffffff,84bb53bffffffff,11.48,"POLYGON ((1737128.831 6058485.634, 1732967.25 ...",0.052
320,85bb5333fffffff,84bb533ffffffff,11.12,"POLYGON ((1752803.747 6065946.671, 1748647.662...",0.064


In [9]:
hr5_cells.to_file('global-inputs/HR5-cells-beach-slope.gpkg')

In [10]:
hr5_cells

,cell_id,parent_id,sediment_area_ha,geometry,beach_slope
0,85bb0b37fffffff,84bb0b3ffffffff,88.52,"POLYGON ((1922896.542 5807892.773, 1918824.379...",0.066
1,85bb56cffffffff,84bb56dffffffff,98.72,"POLYGON ((1895219.978 5828219.058, 1891134.011...",0.062
2,85bb560ffffffff,84bb561ffffffff,33.08,"POLYGON ((1869065.502 5866362.246, 1864965.364...",0.076
3,85bb5293fffffff,84bb529ffffffff,42.80,"POLYGON ((1858357.096 5912132.056, 1854249.225...",0.098
4,85bb0b07fffffff,84bb0b1ffffffff,53.76,"POLYGON ((1950490.925 5787619.369, 1946432.877...",0.055
...,...,...,...,...,...
317,85bb2d37fffffff,84bb2d3ffffffff,7.20,"POLYGON ((1670719.738 5326592.803, 1666569.495...",0.061
318,85bb2d33fffffff,84bb2d3ffffffff,54.52,"POLYGON ((1684781.284 5316739.357, 1680639.31 ...",0.054
319,85bb53affffffff,84bb53bffffffff,11.48,"POLYGON ((1737128.831 6058485.634, 1732967.25 ...",0.052
320,85bb5333fffffff,84bb533ffffffff,11.12,"POLYGON ((1752803.747 6065946.671, 1748647.662...",0.064


In [11]:
sites_df = pd.read_csv('global-inputs/sites.csv')

cell_ids = list(sites_df['cell_id'])

In [39]:
cells = hr5_cells[hr5_cells['cell_id'] == cell_ids[0]]

In [84]:
import glob
from datetime import datetime

def apply_tidal_correction(cell_id):
    slope = hr5_cells.loc[hr5_cells['cell_id'] == cell_id, 'beach_slope'].values[0]
    ref_ele = 0

    folder = glob.glob(f"data/{cell_id}")[0]
    raw_df = pd.read_csv(f"{folder}/cell_timeseries.csv")
    raw_df['date'] = pd.to_datetime(raw_df['date'])

    tide_df = pd.read_json(f"{folder}/image_metadata.json")
    tide_df['date'] = tide_df['image_id'].apply(lambda x:datetime.strptime(x.split('/')[-1][:8],'%Y%m%d'))

    merged_df = pd.merge(raw_df, tide_df[['date', 'tide_level_MSL']], on='date', how='left') # Merge raw_df and tide_df on the date column

    merged_df['tidal_correction'] = (merged_df.tide_level_MSL-ref_ele)/slope
    merged_df['corrected_shoreline_chg'] = merged_df.IW_shoreline_chg + merged_df.tidal_correction

    print(merged_df)


In [85]:
cell_ids = cells.cell_id.tolist()
#print(cell_ids)
for cell in cell_ids:
    #print(cell)
    apply_tidal_correction(cell)

   Unnamed: 0       date  IW_CDI  IW_shoreline_chg  EOV_CDI  \
0         0.0 2023-10-03   -0.91             -9.10    -0.50   
1         NaN 2023-12-07    4.23             42.27     0.70   
2         NaN 2024-01-01   -3.45            -34.49     0.10   
3         NaN 2024-01-21    0.77              7.73    -0.29   
4         NaN 2024-03-11   -4.53            -45.33     0.31   
5         NaN 2024-03-21    2.93             29.30    -0.94   

   EOV_shoreline_chg  IW_area_change (Ha)  EOV_area_change (Ha)  \
0              -4.98               -66.33                -40.69   
1               7.03               135.98                 18.83   
2               0.98               -89.55                  4.29   
3              -2.92               306.45                 -8.52   
4               3.13              -368.76                 12.42   
5              -9.41               125.37                -43.55   

   sand_area (Ha)  water_area (Ha)  vegetation_area (Ha)  tide_level_MSL  \
0          2